In [1]:
import pandas as pd
import nltk
# nltk.download('all')
import re
import numpy as np
import collections
import sklearn as sk
import string
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

In [7]:
df = pd.read_csv("../A_Source_Data/cp2077_reviews.csv.zip", compression="zip")
df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
df["Review"] = df["Review"].astype("str")
df

,Review,Recommended or Not Recommended,Date Timestamp Created
0,"Been here since day 1, and I am someone who ca...",True,1645046263
1,Had to replay this immediately to demolish Ada...,True,1663224196
2,Patch 1.5 fixed everything for me. \nThe quest...,True,1645267750
3,Watches Edgerunners -> Downloads Cyberpunk 207...,True,1667117035
4,I remember hearing about Cyberpunk 2077 around...,True,1664423074
...,...,...,...
16594,boobs :D,True,1608792512
16595,"+ The great main story, cyberpunk atmosphere a...",True,1608792267
16596,If you got the hardware to run this game you d...,True,1608777643
16597,the glitches in my experience have not been en...,True,1608777582


In [8]:
def data_cleaning(raw_data):
    raw_data = raw_data.translate(str.maketrans('', '', string.punctuation + string.digits)) 
    words = raw_data.lower().split() 
    # stops = set(stopwords.words( "english")) 
    stops = set(stopwords)
    # stops = stopwords.words('english')
    useful_words = [w for w in words if not w in stops]
    # use regex to change any word with cyberpunk in it to cyberpunk
    useful_words = [re.sub(r'\b.*cyberpunk.*\b', 'cyberpunk', w) for w in useful_words]
    useful_words = [re.sub(r'c.{6,8}k', 'cyberpunk', w) for w in useful_words]
    useful_words = [w.replace('cyber punk', 'cyberpunk') for w in useful_words]
    useful_words = [w.replace('cyberpunks', 'cyberpunk') for w in useful_words]
    useful_words = [w.replace('cyber punk ', 'cyberpunk') for w in useful_words]
    useful_words = [w.replace('cyberpunk2077', 'cyberpunk') for w in useful_words]
    useful_words = [w.replace('cyberpunk 2077', 'cyberpunk') for w in useful_words]
    useful_words = [w.replace('cyberpunk2077 ', 'cyberpunk') for w in useful_words]
    useful_words = [w.replace('cyberpunk 2077 ', 'cyberpunk') for w in useful_words]
    useful_words = [w.replace('cyberpunk2077game', 'cyberpunk') for w in useful_words]
    useful_words = [w.replace('cyberpunk2077 game', 'cyberpunk') for w in useful_words]
    useful_words = [re.sub(r'[^\w\s]','',w) for w in useful_words]
    return( " ".join(useful_words))

In [9]:
df['Review']=df['Review'].apply(data_cleaning) 

In [10]:
# make a set_column to count the number of words in each review
df["review_len"] = df["Review"].apply(lambda x: len(x.split()))
df

,Review,Recommended or Not Recommended,Date Timestamp Created,review_len
0,day someone came ps version game quite journey...,True,1645046263,259
1,replay immediately demolish adam smasher,True,1663224196,5
2,patch fixed everything quests make sense fixer...,True,1645267750,57
3,watches edgerunners downloads cyberpunk time life,True,1667117035,6
4,remember hearing cyberpunk around announced in...,True,1664423074,253
...,...,...,...,...
16594,boobs d,True,1608792512,2
16595,great main story cyberpunk atmosphere good vis...,True,1608792267,18
16596,got hardware run game definitely,True,1608777643,5
16597,glitches experience enough hold awesomeness ga...,True,1608777582,8


In [23]:
df["review_len"] = df['review_len'].astype(int)

In [24]:
df.dtypes

Review                            object
Recommended or Not Recommended      bool
Date Timestamp Created             int64
review_len                         int64
dtype: object

In [37]:
ast = {}
for x in df['review_len']:
    if x in ast:
        ast[x] += 1
    else:
        ast[x] = 1

ast

{259: 6,
 5: 630,
 57: 58,
 6: 493,
 253: 4,
 4: 669,
 277: 8,
 77: 43,
 74: 56,
 39: 107,
 163: 8,
 127: 36,
 1: 787,
 17: 204,
 647: 1,
 139: 21,
 7: 468,
 50: 64,
 14: 256,
 684: 1,
 231: 7,
 43: 107,
 449: 5,
 473: 1,
 16: 220,
 20: 174,
 695: 2,
 713: 4,
 314: 5,
 674: 7,
 48: 78,
 290: 7,
 690: 3,
 44: 83,
 2: 787,
 334: 4,
 258: 10,
 89: 44,
 3: 713,
 210: 16,
 330: 5,
 274: 4,
 69: 42,
 217: 8,
 595: 2,
 153: 12,
 702: 3,
 65: 53,
 64: 51,
 193: 19,
 252: 7,
 679: 4,
 8: 399,
 425: 2,
 154: 15,
 182: 17,
 111: 22,
 21: 160,
 41: 91,
 11: 313,
 165: 15,
 13: 237,
 354: 5,
 362: 4,
 174: 20,
 9: 322,
 138: 24,
 771: 1,
 125: 24,
 373: 5,
 181: 8,
 714: 3,
 18: 194,
 215: 14,
 591: 2,
 388: 1,
 324: 3,
 323: 5,
 71: 58,
 26: 143,
 73: 38,
 673: 2,
 381: 3,
 606: 3,
 12: 261,
 35: 109,
 90: 29,
 276: 4,
 228: 8,
 142: 13,
 520: 1,
 97: 31,
 741: 2,
 278: 7,
 697: 3,
 249: 9,
 593: 3,
 716: 5,
 53: 64,
 91: 27,
 108: 37,
 131: 16,
 320: 6,
 146: 22,
 15: 232,
 295: 2,
 503: 2,
 33: 